## 유사도 챗봇 성능 향상 기법
🤔 키워드는 잡아내면서 질문의 맥락도 캐치하고 싶다<br>
💡 BM25 + Embedding 모델을 결합하여 유사도를 측정해 보자!

In [ ]:
pip install rank_bm25 konlpy sentence_transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3

### 데이터 불러와서 리스트 생성

In [ ]:
import json

# JSON 파일 경로 지정
file_path = 'science_data.json'

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 질문과 답변 리스트 생성
questions = [item['instruction'] for item in data]
answers = [item['output'] for item in data]

print("총 질문 개수:", len(questions))

총 질문 개수: 68


### BM25 상위 3개 답변 확인

In [ ]:
from rank_bm25 import BM25Okapi
from konlpy.tag import Okt
import numpy as np

# 형태소 분석기를 통한 한국어 토큰화
okt = Okt()
tokenized_answers = [okt.morphs(question) for question in answers]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_answers)

# 사용자 질문 입력
query = "전도가 뭐야?"
tokenized_query = okt.morphs(query)

# BM25 점수 계산 및 가장 관련성 높은 질문 찾기
bm25_scores = bm25.get_scores(tokenized_query)
# best_doc_idx = doc_scores.argmax()

# 상위 3개의 인덱스 찾기
top_3_indices = np.argsort(bm25_scores)[-3:][::-1]

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 질문 및 답변:")
for idx in top_3_indices:
    print(f"질문: {questions[idx]}")
    print(f"답변: {answers[idx]}")
    print()

사용자 질문: 전도가 뭐야?

상위 3개 질문 및 답변:
질문: 열전달의 세 가지 방법을 설명해 주세요.
답변: 열전달의 세 가지 방법은 전도, 대류, 복사입니다. 전도는 열이 물질을 통해 직접 전달되는 과정, 대류는 유체의 이동을 통해 열이 전달되는 과정, 복사는 열이 전자기파의 형태로 전달되는 과정입니다.

질문: 기체의 밀도와 온도 사이의 관계를 설명해 주세요.
답변: 기체의 밀도는 온도에 반비례합니다. 기체의 온도가 상승하면 기체 분자들이 더 빠르게 움직여 부피가 커지므로 밀도가 감소합니다. 반대로 온도가 낮아지면 밀도가 증가합니다.

질문: 화학에서 반응 속도에 영향을 미치는 요소를 설명해 주세요.
답변: 화학 반응 속도에 영향을 미치는 요소로는 온도, 반응물의 농도, 촉매, 압력(기체 반응의 경우) 등이 있습니다. 온도가 높아지면 반응 속도가 증가하며, 농도가 높을수록 반응 속도가 빨라집니다.



### Embedding 모델 상위 3개 답변 확인

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Hugging Face 임베딩 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 질문 임베딩 생성
answer_embeddings = model.encode(answers, convert_to_tensor=True)

# 사용자 질문 입력 및 임베딩 생성
query = "전도가 뭐야?"
query_embedding = model.encode(query, convert_to_tensor=True)

# 코사인 유사도 계산
cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings)

# 상위 3개의 인덱스 찾기
top_3_indices = torch.argsort(cosine_scores, descending=True)[0][:3].tolist()

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 질문 및 답변:")
for idx in top_3_indices:
    print(f"질문: {questions[idx]}")
    print(f"답변: {answers[idx]}")
    print()

사용자 질문: 전도가 뭐야?

상위 3개 질문 및 답변:
질문: 열전달의 세 가지 방법을 설명해 주세요.
답변: 열전달의 세 가지 방법은 전도, 대류, 복사입니다. 전도는 열이 물질을 통해 직접 전달되는 과정, 대류는 유체의 이동을 통해 열이 전달되는 과정, 복사는 열이 전자기파의 형태로 전달되는 과정입니다.

질문: 전기 회로에서 전압의 역할을 설명해 주세요.
답변: 전기 회로에서 전압은 전류를 흐르게 하는 원동력입니다. 전압은 전하를 전도체를 통해 이동시키는 힘을 제공하며, 전기 회로에서 전류의 흐름을 결정합니다.

질문: 전기 회로에서 전류의 정의를 설명해 주세요.
답변: 전류는 전하가 전도체를 통해 흐르는 속도를 나타내는 물리량입니다. 전류의 단위는 암페어(A)이며, 전압에 의해 전하가 이동하면서 회로를 통해 흐릅니다.



### 앙상블 모델 상위 3개 답변 확인

In [ ]:
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
from konlpy.tag import Okt

# JSON 파일 경로 지정
file_path = 'science_data.json'

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 질문과 답변 리스트 생성
questions = [item['instruction'] for item in data]
answers = [item['output'] for item in data]

# 형태소 분석기를 통한 한국어 토큰화
okt = Okt()
tokenized_answers = [okt.morphs(answer) for answer in answers]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_answers, k1=1.5, b=0.75)

# Hugging Face 임베딩 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 답변 임베딩 생성
answer_embeddings = model.encode(answers, convert_to_tensor=True)

# 사용자 질문 입력
query = "전도가 뭐야?"

# BM25 검색
tokenized_query = okt.morphs(query)
bm25_scores = bm25.get_scores(tokenized_query)

# 임베딩 검색
query_embedding = model.encode(query, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings).squeeze().tolist()

# 점수 결합
alpha = 0.5
combined_scores = [(alpha * bm25_score + (1 - alpha) * cosine_score) for bm25_score, cosine_score in zip(bm25_scores, cosine_scores)]

# 상위 3개의 결과 인덱스 찾기
top_3_indices = np.argsort(combined_scores)[::-1][:3]

# 결과 출력
print("검색어:", query)
print("\n상위 3개 질문 및 답변:")
for idx in top_3_indices:
    print(f"질문: {questions[idx]}")
    print(f"답변: {answers[idx]}")
    print(f"BM25 점수: {bm25_scores[idx]}")
    print(f"임베딩 유사도: {cosine_scores[idx]}")
    print(f"결합 점수: {combined_scores[idx]}")
    print()

검색어: 전도가 뭐야?

상위 3개 질문 및 답변:
질문: 열전달의 세 가지 방법을 설명해 주세요.
답변: 열전달의 세 가지 방법은 전도, 대류, 복사입니다. 전도는 열이 물질을 통해 직접 전달되는 과정, 대류는 유체의 이동을 통해 열이 전달되는 과정, 복사는 열이 전자기파의 형태로 전달되는 과정입니다.
BM25 점수: 5.245804106134422
임베딩 유사도: 0.38906535506248474
결합 점수: 2.8174347305984533

질문: 기체의 밀도와 온도 사이의 관계를 설명해 주세요.
답변: 기체의 밀도는 온도에 반비례합니다. 기체의 온도가 상승하면 기체 분자들이 더 빠르게 움직여 부피가 커지므로 밀도가 감소합니다. 반대로 온도가 낮아지면 밀도가 증가합니다.
BM25 점수: 1.6201333766383752
임베딩 유사도: 0.07426438480615616
결합 점수: 0.8471988807222657

질문: 화학에서 반응 속도에 영향을 미치는 요소를 설명해 주세요.
답변: 화학 반응 속도에 영향을 미치는 요소로는 온도, 반응물의 농도, 촉매, 압력(기체 반응의 경우) 등이 있습니다. 온도가 높아지면 반응 속도가 증가하며, 농도가 높을수록 반응 속도가 빨라집니다.
BM25 점수: 1.4968180981192423
임베딩 유사도: 0.1477956622838974
결합 점수: 0.8223068802015698



### 앙상블 모델 Gradio 챗봇 구현
➡ 정확도는 높아졌으나 답변 출력 속도가 느리다 😂

In [ ]:
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import numpy as np
import gradio as gr
from konlpy.tag import Okt
from functools import lru_cache

# 데이터 파일 경로 설정
data_files = {
    '국어': 'korean_data.json',
    '수학': 'math_data.json',
    '영어': 'english_data.json',
    '사회/역사': 'social_data.json',
    '과학': 'science_data.json'
}

# 형태소 분석기와 임베딩 모델 로드
okt = Okt()
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

@lru_cache(maxsize = None)
def load_data(topic):
    # 선택된 주제에 맞는 데이터 파일 로드
    file_path = data_files.get(topic)
    if not file_path:
        return [], [], [], []

    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    questions = [item['instruction'] for item in data]
    answers = [item['output'] for item in data]

    # 형태소 분석을 통한 한국어 토큰화
    tokenized_answers = [okt.morphs(answer) for answer in answers]

    # BM25 모델 생성
    bm25 = BM25Okapi(tokenized_answers, k1=1.5, b=0.75)

    # 답변 임베딩 생성
    answer_embeddings = model.encode(answers, convert_to_tensor=True)

    return questions, answers, bm25, answer_embeddings

def get_answer(query, topic):
    questions, answers, bm25, answer_embeddings = load_data(topic)
    if not questions:
        return "데이터를 로드할 수 없습니다. 주제를 선택하세요."

    # BM25 검색
    tokenized_query = okt.morphs(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    # 임베딩 검색
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings).squeeze().tolist()

    # 점수 결합
    alpha = 0.5
    combined_scores = [(alpha * bm25_score + (1 - alpha) * cosine_score)
                       for bm25_score, cosine_score in zip(bm25_scores, cosine_scores)]

    # 상위 3개의 결과 인덱스 찾기
    top_3_indices = np.argsort(combined_scores)[::-1][:3]

    # 결과 구성
    results = []
    for idx in top_3_indices:
        results.append({
            "질문": questions[idx],
            "답변": answers[idx],
            "BM25 점수": bm25_scores[idx],
            "임베딩 유사도": cosine_scores[idx],
            "결합 점수": combined_scores[idx]
        })

    return results

# Gradio 인터페이스 설정
def gradio_interface(query, topic):
    results = get_answer(query, topic)
    if isinstance(results, str):
        return results
    return "\n".join([f"질문: {result['질문']}\n답변: {result['답변']}\nBM25 점수: {result['BM25 점수']}\n임베딩 유사도: {result['임베딩 유사도']}\n결합 점수: {result['결합 점수']}"
                       for result in results])

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="질문", placeholder="여기에 질문을 입력하세요."),
        gr.Dropdown(choices=list(data_files.keys()), label="주제 선택")
    ],
    outputs="text",
    title="학습 챗봇",
    description="과목을 선택하고 궁금한 걸 물어보세요!"
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ea4502cf1ba1161d7d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 앙상블 모델 Gradio 챗봇 구현 (pkl 파일로 속도 높인 버전)

In [ ]:
import json
import pickle
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from konlpy.tag import Okt

# 데이터 파일 경로 설정
data_files = {
    '국어': 'korean_data.json',
    '수학': 'math_data.json',
    '영어': 'english_data.json',
    '사회역사': 'social_data.json',
    '과학': 'science_data.json'
}

# 형태소 분석기와 임베딩 모델 로드
okt = Okt()
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def save_pickle_file(topic):
    # 선택된 주제에 맞는 데이터 파일 로드
    file_path = data_files.get(topic)
    if not file_path:
        print(f"{topic}의 데이터 파일을 찾을 수 없습니다.")
        return

    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    questions = [item['instruction'] for item in data]
    answers = [item['output'] for item in data]

    # 형태소 분석을 통한 한국어 토큰화
    tokenized_answers = [okt.morphs(answer) for answer in answers]

    # BM25 모델 생성
    bm25 = BM25Okapi(tokenized_answers, k1=1.5, b=0.75)

    # 답변 임베딩 생성
    answer_embeddings = model.encode(answers, convert_to_tensor=True)

    # 피클 파일로 저장
    pickle_file_path = f"{topic}_data.pkl"
    with open(pickle_file_path, 'wb') as f:
        pickle.dump((questions, answers, bm25, answer_embeddings), f)

    print(f"{topic}의 피클 파일이 저장되었습니다: {pickle_file_path}")

# 모든 주제에 대해 피클 파일 저장
for topic in data_files.keys():
    save_pickle_file(topic)

국어의 피클 파일이 저장되었습니다: 국어_data.pkl
수학의 피클 파일이 저장되었습니다: 수학_data.pkl
영어의 피클 파일이 저장되었습니다: 영어_data.pkl
사회역사의 피클 파일이 저장되었습니다: 사회역사_data.pkl
과학의 피클 파일이 저장되었습니다: 과학_data.pkl


In [ ]:
import pickle
import gradio as gr
import numpy as np
from sentence_transformers import SentenceTransformer, util
from konlpy.tag import Okt, Komoran
from functools import lru_cache

# 형태소 분석기와 임베딩 모델 로드
# okt = Okt()
komoran = Komoran()
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

@lru_cache(maxsize=None)
def load_data(topic):
    pickle_file_path = f"{topic}_data.pkl"
    print(f"Trying to load: {pickle_file_path}")
    try:
        with open(pickle_file_path, 'rb') as f:
            questions, answers, bm25, answer_embeddings = pickle.load(f)
        return questions, answers, bm25, answer_embeddings
    except FileNotFoundError:
        return [], [], [], []
    except Exception as e:
        print(f"Error loading data: {e}")
        return [], [], [], []
def get_answer(topic, query):
    questions, answers, bm25, answer_embeddings = load_data(topic)
    if not questions:
        return "데이터를 로드할 수 없습니다. 주제를 선택하세요."

    tokenized_answers = [komoran.morphs(answer) for answer in answers]
    tokenized_query = komoran.morphs(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    # 임베딩 검색
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings).squeeze().tolist()

    # 점수 결합
    alpha = 0.5
    combined_scores = [(alpha * bm25_score + (1 - alpha) * cosine_score)
                       for bm25_score, cosine_score in zip(bm25_scores, cosine_scores)]

    # 상위 3개의 결과 인덱스 찾기
    top_3_indices = np.argsort(combined_scores)[::-1][:3]

    # 결과 구성
    results = []
    for idx in top_3_indices:
        results.append({
            "질문": questions[idx],
            "답변": answers[idx],
            "BM25 점수": bm25_scores[idx],
            "임베딩 유사도": cosine_scores[idx],
            "결합 점수": combined_scores[idx]
        })

    return results

# Gradio 인터페이스 설정
def gradio_interface(topic, query):
    results = get_answer(topic, query)
    if isinstance(results, str):
        return results
    return "\n".join([f"질문: {result['질문']}\n답변: {result['답변']}\nBM25 점수: {result['BM25 점수']}\n임베딩 유사도: {result['임베딩 유사도']}\n결합 점수: {result['결합 점수']}"
                       for result in results])

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(choices=list(data_files.keys()), label="과목 선택"),
        gr.Textbox(label="질문", placeholder="여기에 질문을 입력하세요.")
    ],
    outputs="text",
    title="학습 챗봇",
    description="과목을 선택하고 궁금한 걸 물어보세요!"
)

iface.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ab0e6a8176289b2143.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Trying to load: 수학_data.pkl


/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://ea4502cf1ba1161d7d.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://ab0e6a8176289b2143.gradio.live
